# Klasyfikacja binarna obecności osób w pomieszczeniu na podstawie danych sensorowych (Room Occupancy Estimation) #

* Michał Adamiec 217633
* Piotr Lewandowski 217357

### **Streszczenie**
Celem projektu jest opracowaneie modelu klasyfikacyjnego umożliwiającego wykrycie osób w pomieszczeniu na podstawie danych wielu nieinwazyjnych czujników środowiskowych, takich jak temperatura, światło, dźwięk, CO2 oraz PIR (pasywny czujnik podczerwieni). Zmienna docelowa została przekształcona do postaci binarnej, co pozwala na rozróżnienie sytuacji, gdy pomieszczenie jest puste lub zajęte.

### **Słowa kluczowe**
- klasyfikacja binarna
- analiza danych
- czujniki środowiskowe
- obecność w pomieszczeniu
- uczenie maszynowe

### **Wprowadzenie**
Projekt dotyczy wykrywania obecności osób w pomieszczeniu na podstawie danych z czujników środowiskowych.

### **Przedmiot badania**
Analizujemy dane zebrane w pomieszczeniu wyposażonym w 7 czujników, rejestrujących parametry środowiskowe co 30 sekund.

### **Cel projketu**
Celem jest stworzenie modelu klasyfikacyjnego, który na podstawie danych z czujników przewidzi obecność osób w pomieszczeniu (klasyfikacja binarna).

### **Wstępna analiza danych**

**Charakterystyka zbioru danych**
- Liczba obserwacji: 10129  
- Liczba cech: 18  
- Typ cech: numeryczne (real)

**Przedstawienie dostępnych zmiennych**
* **Date** - data pomiaru w formacie YYYY/MM/DD
* **Time** - godzina pomiaru w formacie HH:MM:SS
* **S1_Temp, S2_Temp, S3_Temp, S4_Temp** - temperatura z czujników S1-S4 (w stopniach Celsjusza)
* **S1_Light, S2_Light, S3_Light, S4_Light** - natężenie światła z czujników S1-S4 (w luksach)
* **S1_Sound, S2_Sound, S3_Sound, S4_Sound** - poziom dźwięku z czujników S1-S4 (w woltach, odczyt z ADC)
* **S5_CO2** - stężenie CO2 z czujnika S5 (w PPM)
* **S5_CO2_Slope** - nachylenie zmian CO2 w oknie czasowym
* **S6_PIR** - detekcja ruchu przez czujnik PIR S6
  * *0* - brak ruchu
  * *1* - wykryto ruch
* **S7_PIR** - detekcja ruchu przez czujnik PIR S7
  * *0* - brak ruchu
  * *1* - wykryto ruch
* **Room_Occupancy_Count** - liczba osób w pomiesczeniu (zmienna docelowa)
  * *0* - pomiesczenie puste
  * *1, 2, 3* - liczba obecnych osób

### Importy

In [130]:
# function that fetches dataset from ucimlrepo
from ucimlrepo import fetch_ucirepo

### Pobranie zbioru danych i informacje o zmiennych

In [131]:
# fetch dataset 
room_occupancy_estimation = fetch_ucirepo(id=864) 
  
# data (as pandas dataframes) 
X = room_occupancy_estimation.data.features 
y = room_occupancy_estimation.data.targets 
  
# metadata 
print(room_occupancy_estimation.metadata) 
  
# variable information 
print(room_occupancy_estimation.variables) 


{'uci_id': 864, 'name': 'Room Occupancy Estimation', 'repository_url': 'https://archive.ics.uci.edu/dataset/864/room+occupancy+estimation', 'data_url': 'https://archive.ics.uci.edu/static/public/864/data.csv', 'abstract': 'Data set for estimating the precise number of occupants in a room using multiple non-intrusive environmental sensors like temperature, light, sound, CO2 and PIR.', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10129, 'num_features': 18, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Room_Occupancy_Count'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2018, 'last_updated': 'Wed Aug 16 2023', 'dataset_doi': '10.24432/C5P605', 'creators': ['Adarsh Pal Singh', 'Sachin Chaudhari'], 'intro_paper': {'ID': 275, 'type': 'NATIVE', 'title': 'Machine Learning-Based Occupancy Estimation Using Multivariate Sensor Nodes', 'auth

### Podgląd danych

In [132]:
X.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0


In [133]:
y.head()

,Room_Occupancy_Count
0,1
1,1
2,1
3,1
4,1


### Pierwsze wystąpienia dla każdej wartości Room_Occupancy_Count

In [134]:
unique_values = y['Room_Occupancy_Count'].unique()
indices = [y[y['Room_Occupancy_Count'] == val].index[0] for val in unique_values]
display(X.loc[indices])

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0
75,2017/12/22,11:28:29,25.38,25.44,24.81,25.69,150,187,63,49,0.46,0.46,0.21,0.11,450,1.088462,1,1
191,2017/12/22,12:30:16,25.69,28.44,25.19,26.00,156,244,190,64,1.12,0.49,0.77,0.25,645,0.411538,1,1
346,2017/12/22,14:15:58,26.19,27.25,26.13,26.44,19,22,97,60,0.42,3.03,3.67,0.64,950,-0.069231,1,1


In [135]:
display(y.loc[indices])

,Room_Occupancy_Count
0,1
75,2
191,3
346,0


### Liczba wystąpień każdej zmiennej Room_Occupancy_Count w zbiorze danych

In [136]:
print(y['Room_Occupancy_Count'].value_counts())

Room_Occupancy_Count
0    8228
2     748
3     694
1     459
Name: count, dtype: int64


## Tworzenie nowych cech

**Nowo utworzone cechy**
* **Temp_mean** – średnia temperatura z czujników S1–S4 (w stopniach Celsjusza)
* **Light_sum** - suma natężenia światła z czterech czujników (w luksach)
* **Sound_mean** - średnia wartość poziomu dźwięku z czterech czujników (w woltach)
* **CO2_to_Sound** - stosunek stężenia CO2 do sumy poziomu dźwięku (bezwymiarowe)
* **PIR_active** - liczba aktywnych czujników ruchu PIR (wartość 0, 1 lub 2)

In [137]:
X['Temp_mean'] = X[['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp']].mean(axis=1)
X['Light_sum'] = X[['S1_Light', 'S2_Light', 'S3_Light', 'S4_Light']].sum(axis=1)
X['Sound_mean'] = X[['S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound']].mean(axis=1)
X['CO2_to_Sound'] = X['S5_CO2'] / (X[['S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound']].sum(axis=1) + 1e-6)
X['PIR_active'] = X['S6_PIR'] + X['S7_PIR']

X.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,...,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Temp_mean,Light_sum,Sound_mean,CO2_to_Sound,PIR_active
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,...,0.06,390,0.769231,0,0,24.9075,248,0.0975,999.997436,0
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,...,0.06,390,0.646154,0,0,24.9225,247,0.2750,354.545132,0
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,...,0.06,390,0.519231,0,0,24.9225,248,0.1700,573.528568,0
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,...,0.09,390,0.388462,0,0,24.9375,248,0.1750,557.142061,0
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,...,0.06,390,0.253846,0,0,24.9375,249,0.0900,1083.330324,0
